In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import paddle
from paddle.io import Dataset
from paddle.vision.transforms import transforms
from paddle.vision.models import resnet18
from paddle.nn import functional as F
print(paddle.__version__)


2.3.0


In [2]:
Train_Dir = './data/training.csv'
Test_Dir = './data/test.csv'
lookid_dir = './data/IdLookupTable.csv'

In [3]:
class ImgTransforms(object):
    '''
    图像预处理工具，用于将图像进行升维(96, 96) => (96, 96, 3),
    并对图像的维度进行转换从HWC变为CHW
    '''
    def __init__(self, fmt):
        self.format = fmt

    def __call__(self, img):
        if len(img.shape) == 2:
            img = np.expand_dims(img, axis=2)
        img =  img.transpose(self.format)

        if img.shape[0] == 1:
            img = np.repeat(img, 3, axis=0)
        return img


In [8]:
class FaceDataset(Dataset):
    def __init__(self, data_path, mode='train', val_split=0.2):
        self.mode = mode
        assert self.mode in ['train', 'val', 'test'], \
            "mode should be 'train' or 'test', but got {}".format(self.mode)
        self.data_source = pd.read_csv(data_path)
        # 清洗数据, 数据集中有很多样本只标注了部分关键点, 这里有两种策略
        # 第一种, 将未标注的位置从上一个样本对应的关键点复制过来
        # self.data_source.fillna(method = 'ffill',inplace = True)
        # 第二种, 将包含有未标注的样本从数据集中移除
        self.data_source.dropna(how="any", inplace=True)  
        self.data_label_all = self.data_source.drop('Image', axis = 1)
        
        # 划分训练集和验证集合
        if self.mode in ['train', 'val']:
            np.random.seed(43)
            data_len = len(self.data_source)
            # 随机划分
            shuffled_indices = np.random.permutation(data_len)
            # 顺序划分
            # shuffled_indices = np.arange(data_len)
            self.shuffled_indices = shuffled_indices
            val_set_size = int(data_len*val_split)
            if self.mode == 'val':
                val_indices = shuffled_indices[:val_set_size]
                self.data_img = self.data_source.reindex().iloc[val_indices]
                self.data_label = self.data_label_all.reindex().iloc[val_indices]
            elif self.mode == 'train':
                train_indices = shuffled_indices[val_set_size:]
                self.data_img = self.data_source.reindex().iloc[train_indices]
                self.data_label = self.data_label_all.reindex().iloc[train_indices]
        elif self.mode == 'test':
            self.data_img = self.data_source
            self.data_label = self.data_label_all

        self.transforms = transforms.Compose([
            ImgTransforms((2, 0, 1))
        ])

    # 每次迭代时返回数据和对应的标签
    def __getitem__(self, idx):

        img = self.data_img['Image'].iloc[idx].split(' ')
        img = ['0' if x == '' else x for x in img]
        img = np.array(img, dtype = 'float32').reshape(96, 96)
        img = self.transforms(img)
        label = np.array(self.data_label.iloc[idx,:],dtype = 'float32')/96
        return img, label

    # 返回整个数据集的总数
    def __len__(self):
        return len(self.data_img)

In [7]:
# 训练数据集和验证数据集
train_dataset = FaceDataset(Train_Dir, mode='train')
val_dataset = FaceDataset(Train_Dir, mode='val')

# 测试数据集
test_dataset = FaceDataset(Test_Dir,  mode='test')

print(train_dataset)

train


In [ ]:
def plot_sample(x, y, axis):
    img = x.reshape(96, 96)
    axis.imshow(img, cmap='gray')
    axis.scatter(y[0::2], y[1::2], marker='x', s=10, color='b')

fig = plt.figure(figsize=(10, 7))
fig.subplots_adjust(
    left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# 随机取16个样本展示
for i in range(16):
    axis = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    idx = np.random.randint(train_dataset.__len__())
    # print(idx)
    img, label = train_dataset[idx]
    label = label*96
    plot_sample(img[0], label, axis)
plt.show()


In [11]:
class FaceNet(paddle.nn.Layer):
    def __init__(self, num_keypoints, pretrained=False):
        super(FaceNet, self).__init__()
        self.backbone = resnet18(pretrained)
        self.outLayer1 = paddle.nn.Sequential(
            paddle.nn.Linear(1000, 512),
            paddle.nn.ReLU(),
            paddle.nn.Dropout(0.1))
        self.outLayer2 = paddle.nn.Linear(512, num_keypoints*2)
    def forward(self, inputs):
        print("forward")
        print(inputs)
        out = self.backbone(inputs)
        out = self.outLayer1(out)
        out = self.outLayer2(out)
        return out


In [12]:
from paddle.static import InputSpec

num_keypoints = 15
model = paddle.Model(FaceNet(num_keypoints))
model.summary((1,3, 96, 96))


forward
Tensor(shape=[1, 3, 96, 96], dtype=float32, place=Place(cpu), stop_gradient=True,
       [[[[0.93391484, 0.87399602, 0.34922174, ..., 0.11524644,
           0.76829731, 0.71494907],
          [0.65204245, 0.44626629, 0.28582275, ..., 0.39551812,
           0.43062973, 0.48393878],
          [0.78872502, 0.25380990, 0.40041348, ..., 0.12269991,
           0.27579394, 0.70870024],
          ...,
          [0.61812091, 0.51787388, 0.74441856, ..., 0.29643437,
           0.94886225, 0.63021678],
          [0.82991761, 0.96159208, 0.81743032, ..., 0.92363167,
           0.51960462, 0.73603690],
          [0.45481512, 0.11282970, 0.25422928, ..., 0.66255701,
           0.76024348, 0.75224495]],

         [[0.27751577, 0.64261627, 0.15084772, ..., 0.30424410,
           0.51075912, 0.00359466],
          [0.34566522, 0.94111931, 0.86156219, ..., 0.80104059,
           0.83927649, 0.55292875],
          [0.41279531, 0.10739559, 0.09021059, ..., 0.75234067,
           0.25003272, 0.1814

{'total_params': 12227014, 'trainable_params': 12207814}

In [ ]:
model = paddle.Model(FaceNet(num_keypoints=15))
optim = paddle.optimizer.Adam(learning_rate=1e-3,
    parameters=model.parameters())
model.prepare(optim, paddle.nn.MSELoss())
# model.fit(train_dataset, val_dataset, epochs=60, batch_size=256)



In [16]:
for i in range(train_dataset.__len__()):
    print(train_dataset.__getitem__(i))

(array([[[124., 113., 151., ...,  84., 112., 132.],
        [109., 133., 167., ...,  63.,  99., 124.],
        [126., 139., 191., ..., 141.,  63., 111.],
        ...,
        [146., 158., 207., ..., 115., 123., 147.],
        [158., 149., 198., ..., 117., 139., 137.],
        [161., 151., 185., ..., 113., 144., 122.]],

       [[124., 113., 151., ...,  84., 112., 132.],
        [109., 133., 167., ...,  63.,  99., 124.],
        [126., 139., 191., ..., 141.,  63., 111.],
        ...,
        [146., 158., 207., ..., 115., 123., 147.],
        [158., 149., 198., ..., 117., 139., 137.],
        [161., 151., 185., ..., 113., 144., 122.]],

       [[124., 113., 151., ...,  84., 112., 132.],
        [109., 133., 167., ...,  63.,  99., 124.],
        [126., 139., 191., ..., 141.,  63., 111.],
        ...,
        [146., 158., 207., ..., 115., 123., 147.],
        [158., 149., 198., ..., 117., 139., 137.],
        [161., 151., 185., ..., 113., 144., 122.]]], dtype=float32), array([0.71446466, 0

In [ ]:
result = model.predict(val_dataset, batch_size=1)


In [ ]:
def plot_sample(x, y, axis, gt=[]):
    img = x.reshape(96, 96)
    axis.imshow(img, cmap='gray')
    axis.scatter(y[0::2], y[1::2], marker='x', s=10, color='r')
    if gt!=[]:
        axis.scatter(gt[0::2], gt[1::2], marker='x', s=10, color='lime')


fig = plt.figure(figsize=(10, 7))
fig.subplots_adjust(
    left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(16):
    axis = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    idx = np.random.randint(val_dataset.__len__())
    img, gt_label = val_dataset[idx]
    gt_label = gt_label*96
    label_pred = result[0][idx].reshape(-1)
    label_pred = label_pred*96
    plot_sample(img[0], label_pred, axis, gt_label)
plt.show()


In [ ]:
result = model.predict(test_dataset, batch_size=1)


In [ ]:
fig = plt.figure(figsize=(10, 7))
fig.subplots_adjust(
    left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(16):
    axis = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    idx = np.random.randint(test_dataset.__len__())
    img, _ = test_dataset[idx]
    label_pred = result[0][idx].reshape(-1)
    label_pred = label_pred*96
    plot_sample(img[0], label_pred, axis)
plt.show()


In [ ]:
model.save("./model/facial")